In [1]:
import cv2
import time

In [2]:
video=cv2.VideoCapture('Traffic_Laramie_1.mp4')

# Check if the video opened successfully
if (video.isOpened()== False): 
    print("Error opening video file")
    
#detect objects that are being captured
detect_objs = cv2.createBackgroundSubtractorMOG2(varThreshold=100,detectShadows=False)

#list to store the number of detected cars
detected_cars = []
car_counter = 0

In [3]:
#read until video is completed or we press 'Q'
while True:    
    #capture frame-by-frame
    check, frame = video.read()
    #if we reach the end of the video, check will be false
    if not check:
            break
    #define 'main street''s dimension
    main_street = frame[300:400,100:600]
    
    #capture objects in each frame of main street
    foreground_mask  = detect_objs.apply(main_street)
    
    #get contours from the foreground_mask
    contours,hier = cv2.findContours(foreground_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #draw each detect contours green
    green = (0,255,0)
    
    cv2.rectangle(main_street,(12,60),(20,75),green,2)
    
    current_car_positions = [] 
    for x in contours:
        contour_area = cv2.contourArea(x)
        if(contour_area>2500):
            #draw box for detected contours
            x,y,width,height = cv2.boundingRect(x)
            cv2.rectangle(main_street,(x,y),(x+width,y+height),green,2)
            detected_cars.append([x,y])
             
    if check == True:
        cv2.imshow("Main_street_1.1",frame)
        #cv2.imshow("Frame Differencing",foreground_mask)
    
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
            
#after the loop release the video object
video.release()
#destroy all the windows
cv2.destroyAllWindows()

In [4]:
print(car_counter)

0
